In [1]:
from pyspark.sql.types import StructType, StringType, DoubleType, TimestampType
import os
from pyspark.sql.functions import from_json, expr
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.spark:spark-avro_2.12:3.3.1 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark = SparkSession \
    .builder \
    .appName("Spark-Jupyter") \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a3758fea-1a3e-4dc3-9edc-31ec0b466d69;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.common

24/05/21 06:42:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
schema_energy = StructType().add("SolarPower", DoubleType())

In [4]:
df_kafka_raw_energy = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092,broker:29092") \
    .option("subscribe", "energy_data") \
    .option("startingOffsets", "earliest") \
    .option("checkpointLocation", "checkpoint") \
    .load()

# Parse JSON and select only SolarPower
df_parsed_energy = df_kafka_raw_energy \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json("value", schema_energy).alias("data")) \
    .select("data.SolarPower")

# Print to console (only SolarPower)
# df_parsed_energy.writeStream.format("console").start().awaitTermination()

In [ ]:
# Function to Update Metric (simplified)
from prometheus_client import start_http_server, Gauge
solar_power_gauge = Gauge('solar_power_watts', 'Current solar power generation')  # Create gauge here
def process_batch(batch_df, batch_id):    
    # solar_power_gauge.set(5)  # Set the gauge to the constant value 5
    # print(f"Updated solar_power_watts: {solar_power_gauge._value.get()}")
    
    latest_row = batch_df.first()
    solar_power_value = latest_row["SolarPower"] 
    solar_power_gauge.set(solar_power_value)
    print("SolarPower:", solar_power_value)

# Start Metrics Server (Important!)
start_http_server(8000)

# Spark Structured Streaming (Trigger Every Minute)
query = (
    df_parsed_energy.writeStream
    .foreachBatch(process_batch)
    .trigger(processingTime='2 seconds')  
    .start()
)

query.awaitTermination()

24/05/21 06:42:56 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-478056a4-3eb9-4468-9bf2-96ab46fc7c27. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/05/21 06:42:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


24/05/21 06:42:58 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-475479f2-e2d8-4efd-b106-017e7e3f9c97--1688593392-executor-2, groupId=spark-kafka-source-475479f2-e2d8-4efd-b106-017e7e3f9c97--1688593392-executor] Connection to node -1 (localhost/127.0.0.1:9092) could not be established. Broker may not be available.
24/05/21 06:42:58 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-475479f2-e2d8-4efd-b106-017e7e3f9c97--1688593392-executor-2, groupId=spark-kafka-source-475479f2-e2d8-4efd-b106-017e7e3f9c97--1688593392-executor] Bootstrap broker localhost:9092 (id: -1 rack: null) disconnected


SolarPower: 917.210022
24/05/21 06:42:59 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 2000 milliseconds, but spent 2798 milliseconds
SolarPower: 1533.390015
SolarPower: 1533.390015


SolarPower: 1533.390015
SolarPower: 1533.390015
SolarPower: 1561.25
SolarPower: 1561.25
SolarPower: 1561.25
SolarPower: 1561.25
SolarPower: 1561.25
SolarPower: 1561.25
SolarPower: 1571.140015


SolarPower: 1571.140015


...

SolarPower: 1823.839966


SolarPower: 1823.839966
SolarPower: 1823.839966
SolarPower: 1823.839966
SolarPower: 1823.839966
SolarPower: 1835.439941
SolarPower: 1835.439941
SolarPower: 1835.439941
SolarPower: 1835.439941
SolarPower: 1835.439941
SolarPower: 1835.439941
SolarPower: 1840.790039
SolarPower: 1840.790039
SolarPower: 1840.790039
SolarPower: 1840.790039
SolarPower: 1840.790039
SolarPower: 1840.790039
SolarPower: 1854.719971
SolarPower: 1854.719971
SolarPower: 1854.719971


SolarPower: 1854.719971


SolarPower: 1854.719971
SolarPower: 1854.719971
SolarPower: 1857.560059
SolarPower: 1857.560059
SolarPower: 1857.560059
SolarPower: 1857.560059
SolarPower: 1857.560059
SolarPower: 1857.560059
SolarPower: 1864.319946
SolarPower: 1864.319946
SolarPower: 1864.319946
SolarPower: 1864.319946
SolarPower: 1864.319946
SolarPower: 1864.319946
SolarPower: 1876.51001
SolarPower: 1876.51001
SolarPower: 1876.51001
SolarPower: 1876.51001


SolarPower: 1876.51001
SolarPower: 1876.51001
SolarPower: 1893.0
SolarPower: 1893.0
SolarPower: 1893.0
SolarPower: 1893.0
SolarPower: 1893.0
SolarPower: 1893.0
SolarPower: 1885.459961
SolarPower: 1885.459961
SolarPower: 1885.459961
SolarPower: 1885.459961
SolarPower: 1885.459961
SolarPower: 1885.459961
SolarPower: 1889.219971
